In [ ]:
import os
import ast
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
project_path = "..\\Thesis_Code"
os.chdir(project_path)

In [ ]:
def create_images_of_maps():
    start_nodes = [[0, 1, 2, 3, 4, 5, 6, 7, 14, 15, 16, 17, 18, 19, 20, 21], # hw
                   [0, 1, 2, 3, 4, 5, 6, 7, 14, 15, 16, 17, 18, 19, 20, 21], # hw e
                   [0, 1, 4, 5, 10, 11, 14, 15, 20, 21, 24, 25, 30, 31, 34, 35], # r
                   [0, 1, 4, 5, 12, 13, 16, 17, 24, 25, 28, 29, 36, 37, 40, 41], # i
                   [0, 1, 2, 3, 4, 5, 6, 14, 15, 16, 17, 18, 19, 20, 28, 29]] # hw n
    goal_nodes = [[13, 27], # hw
                  [13, 27, 31], # hw e
                  [9, 19, 29, 39], # r
                  [9, 11, 21, 23, 33, 35, 45, 47], # i
                  [13, 27]] # hw n
    
    csv_file_names = ["/highway.csv", "/exit.csv", "/roundabout.csv", "/intersection.csv", "/entrance.csv"]
    img_file_names = ["/highway", "/exit", "/roundabout", "/intersection", "/entrance"]
    
    for s, g, f_csv, f_img in zip(start_nodes, goal_nodes, csv_file_names, img_file_names):
        
        csv_file_path = "./MAPF_Framework/solver/data_exp1/maps" + f_csv
        df = pd.read_csv(csv_file_path)
    
        G = nx.DiGraph()
    
        positions = {}
        edges = []
    
        for i, row in df.iterrows():
            positions[i] = (row["x"],row["y"])
            
            ids = ast.literal_eval(row["neighbours"])
            
            if(type(ids) is int):
                edges.append((row["id"], ids))
            else:
                ids_list = list(ids)
                for el in ids_list:
                    edges.append((row["id"], el))
        
        G.add_nodes_from(positions.keys())
        G.add_edges_from(edges)
        
        for n, p in positions.items():
            G.nodes[n]['pos'] = p
        
        set_node_color = ["#66B3FF" if node in s else "#FF4D4D" if node in g else "#808080"
                         for node in positions.keys()]
        
        plt.figure(figsize=(25,25))
        plt.axis('equal')
        
        nx.draw(G, pos=positions, with_labels=True, node_color=set_node_color, node_size=300)
        ax = plt.gca()
        ax.collections[0].set_edgecolor("#000000") # so that nodes have black edges
        
        #png_file_path = "./img_data/maps"
        png_file_path = "./CNN_Classifier/selector/data_exp1/maps"
        #if not os.path.isdir(png_file_path):
        #    os.makedirs(png_file_path)
            
        extension = ".pdf"
        
        if os.path.isfile(png_file_path+f_img+extension):
            os.remove(png_file_path+f_img+extension)
        
        plt.savefig(png_file_path+f_img+extension, bbox_inches='tight', pad_inches = 0)
        plt.close()
    
    #plt.show()

In [ ]:
create_images_of_maps()

In [ ]:
def get_graphs_of_maps():
    csv_file_names = ["/highway.csv", "/exit.csv", "/roundabout.csv", "/intersection.csv", "/entrance.csv"]
    graphs = []
    node_positions = []
    
    for f_csv in csv_file_names:
        
        #csv_file_path = "./csv_data/maps" + f_csv
        csv_file_path = "./MAPF_Framework/solver/data_exp1/maps" + f_csv
        df = pd.read_csv(csv_file_path)
    
        G = nx.DiGraph()
    
        positions = {}
        edges = []
    
        for i, row in df.iterrows():
            positions[i] = (row["x"],row["y"])
            
            ids = ast.literal_eval(row["neighbours"])
            
            if(type(ids) is int): # (a)
                edges.append((row["id"], ids))
            else:
                ids_list = list(ids) # (), (a,b), (a,b,c)
                for el in ids_list:
                    edges.append((row["id"], el))
        
        G.add_nodes_from(positions.keys())
        G.add_edges_from(edges)
        
        for n, p in positions.items():
            G.nodes[n]['pos'] = p
        
        graphs.append(G)
        node_positions.append(positions)
        
    return graphs, node_positions

In [ ]:
def create_img_dataset():
    
    data_folders = ["/train", "/validation"]
    class_folders = ["/classA/", "/classB/"]
    
    graphs, node_positions = get_graphs_of_maps()
    
    for dataf in data_folders:
        for classf in class_folders:
            #csv_train_path = "./csv_data/train" + classf
            
            csv_path = "./MAPF_Framework/solver/data_n" + dataf + classf
            
            #img_train_path = "./img_data/train" + classf
            img_path = "./CNN_Classifier/selector/data_n" + dataf + classf
            
            if not os.path.isdir(img_path):
                    os.makedirs(img_path)
                    
            print(dataf, classf)
            
            csv_file_names = [f for f in os.listdir(csv_path)]
            
            for f_csv in csv_file_names:
                csv_file_path = csv_path + f_csv
                #print(csv_file_path)
                #break
                df = pd.read_csv(csv_file_path)
        
                map_type = f_csv[5] #:-7] # slice "0003_h_02" -> "h"
            
                #print(f_csv)
                
                if map_type == "h":
                    G = graphs[0]
                    positions = node_positions[0]
                elif map_type == "e":
                    G = graphs[1]
                    positions = node_positions[1]
                elif map_type == "r":
                    G = graphs[2]
                    positions = node_positions[2]
                elif map_type == "i":
                    G = graphs[3]
                    positions = node_positions[3]
                elif map_type == "n":
                    G = graphs[4]
                    positions = node_positions[4]
                
                start_nodes = []
                goal_nodes = []
                
                for i, row in df.iterrows():
                    start_nodes.append(row["start"])
                    goal_nodes.append(row["goal"])
                
                set_node_color = ["#77FF33" if node in start_nodes else "#FF4D4D" if node in goal_nodes else "#808080"
                                 for node in positions.keys()]
                
                plt.figure(figsize=(25,25))
                plt.axis('equal')
                
                nx.draw(G, pos=positions, with_labels=False, node_color=set_node_color, node_size=300)
                ax = plt.gca()
                ax.collections[0].set_edgecolor("#000000") # so that nodes have black edges
                
                f_name = f_csv[:-4] # remove ".csv" ending
                extension = ".png"
                f_img = img_path + f_name + extension
                
                if os.path.isfile(f_img):
                    os.remove(f_img)
                    #print("removed")
                
                plt.savefig(f_img)
                plt.close()

In [ ]:
create_img_dataset()

In [ ]:
def create_test_img_dataset():
    
    #data_folders = ["/train", "/validation"]
    #class_folders = ["/classA/", "/classB/"]
    
    dataf = "/test/"
    
    graphs, node_positions = get_graphs_of_maps()
    
    #for dataf in data_folders:
        #for classf in class_folders:
            #csv_train_path = "./csv_data/train" + classf
            
    csv_path = "./MAPF_Framework/solver/data_n" + dataf
            
            #img_train_path = "./img_data/train" + classf
    img_path = "./CNN_Classifier/selector/data_n" + dataf
            
    if not os.path.isdir(img_path):
        os.makedirs(img_path)
            
    csv_file_names = [f for f in os.listdir(csv_path)]
            
    for f_csv in csv_file_names:
        csv_file_path = csv_path + f_csv
                #print(csv_file_path)
                #break
        df = pd.read_csv(csv_file_path)
        
        map_type = f_csv[5] #:-7] # slice "0003_h_02" -> "h"
            
        #print(map_type)
                
        if map_type == "h":
            G = graphs[0]
            positions = node_positions[0]
        elif map_type == "e":
            G = graphs[1]
            positions = node_positions[1]
        elif map_type == "r":
            G = graphs[2]
            positions = node_positions[2]
        elif map_type == "i":
            G = graphs[3]
            positions = node_positions[3]
        elif map_type == "n":
            G = graphs[4]
            positions = node_positions[4]
                
        start_nodes = []
        goal_nodes = []
                
        for i, row in df.iterrows():
            start_nodes.append(row["start"])
            goal_nodes.append(row["goal"])
                
        set_node_color = ["#77FF33" if node in start_nodes else "#FF4D4D" if node in goal_nodes else "#808080"
                          for node in positions.keys()]
                
        plt.figure(figsize=(25,25))
        plt.axis('equal')
                
        nx.draw(G, pos=positions, with_labels=False, node_color=set_node_color, node_size=300)
        ax = plt.gca()
        ax.collections[0].set_edgecolor("#000000") # so that nodes have black edges
                
        f_name = f_csv[:-4] # remove ".csv" ending
        extension = ".png"
        f_img = img_path + f_name + extension
                
        if os.path.isfile(f_img):
            os.remove(f_img)
                    #print("removed")
                
        plt.savefig(f_img)
        plt.close()

In [ ]:
create_test_img_dataset()

In [ ]:
def analyze_intersection_map():
    path_nodes = [0, 1, 2, 3, 48, 49, 50, 51, 20,
                  4, 5, 6, 7, 52, 53, 54, 55, 22,
                  64, 67, 62, 46,
                  32]
    goal_nodes = [21, 23, 33, 47]
    
    csv_file_name = "/intersection.csv"
    img_file_name = "/intersection"
    
        

    csv_file_path = "./MAPF_Framework/solver/data/maps" + csv_file_name
    df = pd.read_csv(csv_file_path)
    
    G = nx.DiGraph()
    
    positions = {}
    edges = []
    
    for i, row in df.iterrows():
        positions[i] = (row["x"],row["y"])
            
        ids = ast.literal_eval(row["neighbours"])
            
        if(type(ids) is int):
            edges.append((row["id"], ids))
        else:
            ids_list = list(ids)
            for el in ids_list:
                edges.append((row["id"], el))
        
    G.add_nodes_from(positions.keys())
    G.add_edges_from(edges)
        
    for n, p in positions.items():
        G.nodes[n]['pos'] = p
        
    set_node_color = ["#FF950A" if node in path_nodes else "#FF4D4D" if node in goal_nodes else "#808080"
                      for node in positions.keys()]
        
    plt.figure(figsize=(25,25))
    plt.axis('equal')
        
    nx.draw(G, pos=positions, with_labels=True, node_color=set_node_color, node_size=300)
    ax = plt.gca()
    ax.collections[0].set_edgecolor("#000000") # so that nodes have black edges
        
        #png_file_path = "./img_data/maps"
    png_file_path = "./CNN_Classifier/selector/data/maps"
        #if not os.path.isdir(png_file_path):
        #    os.makedirs(png_file_path)
            
    extension = "_path_highlight.pdf"
        
    if os.path.isfile(png_file_path+img_file_name+extension):
        os.remove(png_file_path+img_file_name+extension)
        
    plt.savefig(png_file_path+img_file_name+extension, bbox_inches='tight', pad_inches = 0)
    plt.close()
    
    #plt.show()

In [ ]:
def create_image_of_example():
    start_nodes = [[0, 3]]
    goal_nodes = [[2, 4]]
    
    csv_file_names = ["/example.csv"]
    img_file_names = ["/example"]
    
    for s, g, f_csv, f_img in zip(start_nodes, goal_nodes, csv_file_names, img_file_names):
        
        #csv_file_path = "./csv_data/maps" + f_csv
        csv_file_path = "./MAPF_Framework/solver/data/maps" + f_csv
        df = pd.read_csv(csv_file_path)
    
        G = nx.DiGraph()
    
        positions = {}
        edges = []
    
        for i, row in df.iterrows():
            positions[i] = (row["x"],row["y"])
            
            ids = ast.literal_eval(row["neighbours"])
            
            if(type(ids) is int):
                edges.append((row["id"], ids))
            else:
                ids_list = list(ids)
                for el in ids_list:
                    edges.append((row["id"], el))
        
        G.add_nodes_from(positions.keys())
        G.add_edges_from(edges)
        
        for n, p in positions.items():
            G.nodes[n]['pos'] = p
        
        set_node_color = ["#77FF33" if node in s else "#FF4D4D" if node in g else "#808080"
                         for node in positions.keys()]
        
        plt.figure(figsize=(10,10))
        plt.axis('equal')
        
        nx.draw(G, pos=positions, with_labels=True, node_color=set_node_color, node_size=3000, font_size= 30)
        ax = plt.gca()
        ax.collections[0].set_edgecolor("#000000") # so that nodes have black edges
        
        #png_file_path = "./img_data/maps"
        png_file_path = "./CNN_Classifier/selector/data/maps"
        #if not os.path.isdir(png_file_path):
        #    os.makedirs(png_file_path)
            
        extension = ".pdf"
        
        if os.path.isfile(png_file_path+f_img+extension):
            os.remove(png_file_path+f_img+extension)
        
        plt.savefig(png_file_path+f_img+extension, bbox_inches='tight', pad_inches = 0)
        plt.close()
    
    #plt.show()

In [ ]:
#analyze_intersection_map()
create_image_of_example()

In [ ]:
def analyze_roundabout_map():
    path_nodes = [0, 1, 2, 3, 55, 40, 41, 28, 42, 43, 44, 45, 18, 46, 47, 48, 49, 38, 50, 51, 52, 53, 8,
                  4, 5, 6, 7, 54, 56, 57, 58, 59, 60, 61, 62, 63]
    goal_nodes = [9, 19, 29, 39]
    
    csv_file_name = "/roundabout.csv"
    img_file_name = "/roundabout"
    
        

    csv_file_path = "./MAPF_Framework/solver/data_exp1/maps" + csv_file_name
    df = pd.read_csv(csv_file_path)
    
    G = nx.DiGraph()
    
    positions = {}
    edges = []
    
    for i, row in df.iterrows():
        positions[i] = (row["x"],row["y"])
            
        ids = ast.literal_eval(row["neighbours"])
            
        if(type(ids) is int):
            edges.append((row["id"], ids))
        else:
            ids_list = list(ids)
            for el in ids_list:
                edges.append((row["id"], el))
        
    G.add_nodes_from(positions.keys())
    G.add_edges_from(edges)
        
    for n, p in positions.items():
        G.nodes[n]['pos'] = p
        
    set_node_color = ["#FF950A" if node in path_nodes else "#FF4D4D" if node in goal_nodes else "#808080"
                      for node in positions.keys()]
        
    plt.figure(figsize=(25,25))
    plt.axis('equal')
        
    nx.draw(G, pos=positions, with_labels=True, node_color=set_node_color, node_size=300)
    ax = plt.gca()
    ax.collections[0].set_edgecolor("#000000") # so that nodes have black edges
        
        #png_file_path = "./img_data/maps"
    png_file_path = "./CNN_Classifier/selector/data_exp1/maps"
        #if not os.path.isdir(png_file_path):
        #    os.makedirs(png_file_path)
            
    extension = "_path_highlight.pdf"
        
    if os.path.isfile(png_file_path+img_file_name+extension):
        os.remove(png_file_path+img_file_name+extension)
        
    plt.savefig(png_file_path+img_file_name+extension, bbox_inches='tight', pad_inches = 0)
    plt.close()
    
    #plt.show()

In [ ]:
analyze_roundabout_map()